In [1]:
import os
import convokit
print(convokit.__file__)

/home/lucas/school/6742/code_switching/convokit/__init__.py


In [2]:
from convokit import Corpus, download
from collections import defaultdict
import polyglot
from polyglot.text import Detector
import gc
import json

In [3]:
# Example for how to use Detector
detector = Detector("Something beyond Biddle, please, 中文测试 since he does recognize the validity of Jackson's position regarding Nimitz, even though it he argues that it opened the trials up to ''tu quoque'' criticism. ")
for language in detector.languages:
    print(language)

name: English     code: en       confidence:  92.0 read bytes:  1113
name: Chinese     code: zh       confidence:   6.0 read bytes:  1890
name: un          code: un       confidence:   0.0 read bytes:     0


In [4]:
# Takes in corpus and outputs a dictionary with key measures about code switching within it, including
# how effective the detector was on the whole corpus and which conversations have code switching.
def run_detector_on_corpus(wikiconv_YEAR: Corpus) -> dict:
    utt_count, cs_count, en_count, detector_failed, detector_not_failed = 0, 0, 0, 0, 0
    cs_dict = defaultdict(int)
    cs_convs = {}
    for utt in wikiconv_YEAR.iter_utterances():
        utt_count += 1
        try:
            detector = Detector(utt.text)
            detector_not_failed += 1
            languages = [language.name for language in detector.languages if language.confidence > 0]
            if len(languages) > 1:
                cs_count += 1
                if languages[0] == 'English':
                    en_count += 1
                    for switched in languages[1:]:
                        cs_dict[switched] += 1
                    if utt.root not in cs_convs:
                        lang = defaultdict(int)
                        for switched in languages[1:]:
                            lang[switched] = 1
                        cs_convs[utt.root] = lang
                    else:
                        for switched in languages[1:]:
                            cs_convs[utt.root][switched] += 1
        except:
            detector_failed += 1
            
    res = {}
    res["utt_count"] = utt_count
    res["cs_count"] = cs_count
    res["en_count"] = en_count
    res["detector_failed"] = detector_failed
    res["detector_not_failed"] = detector_not_failed
    res["cs_dict"] = cs_dict
    res["cs_convs"] = cs_convs
    return res

In [5]:
def print_parsed_corpus_stats(year: str, detector_results: dict) -> None:
    print("------------------------------YEAR: " + year + "------------------------------")
    print("Total number of utterances in " + year + ": " + str(detector_results["utt_count"]))
    print("Total number of utterances with two or more languages: " + str(detector_results["cs_count"]))
    print("Total number of utterances with two or more languages, for which English is the main language: " + str(detector_results["en_count"]))
    print("\n")
    print("Percentage of utterances on which polyglot Detector ran successfully: " + str(round(100 * detector_results["detector_not_failed"] / (detector_results["detector_failed"] + detector_results["detector_not_failed"]), 2)) + "%")
    print("Percentage of utterances for which English is the main language in a comment with 2 or more languages: " + str(round(100 * detector_results["en_count"] / detector_results["utt_count"], 4)) + "%")
    print("\n")
    print("Number of 'code-switching' instances per language: ")
    for k, v in detector_results["cs_dict"].items():
        print(k + ": " + str(v))
    print("\n")

In [6]:
def get_cs_convs_topics(corpus: Corpus, cs_convs: dict) -> dict:
    topics = {}
    for root in list(cs_convs.keys()):
        topics[corpus.conversations[root].meta['page_title']] = cs_convs[root]
    return topics

In [7]:
# Produces the dictionary for a particular year
def produce_year_dict(year: str) -> dict:
    
    gc.collect()             # helps manage crowded memory after processing a corpus
    
    print("Opening corpus for year " + year + "...")
    wikiconv_year = Corpus(filename=download("wikiconv-" + year))
    
    print("Running detector on year " + year + "...")
    detector_results = run_detector_on_corpus(wikiconv_year)
    detector_results["topics"] = get_cs_convs_topics(wikiconv_year, detector_results["cs_convs"])    
    
    return detector_results

# Outputs a dictionary to json. Note: make sure you have a local polyglot_results/ directory, otherwise
# this function will fail. 
def output_dict_to_json(year: str, results: dict) -> None:
    filepath = "./polyglot_results/" + year + "_results.json"
    with open(filepath, "w") as f:
        json.dump(results, f)  
    print("Successfully output year " + year + " dictionary to " + filepath)

In [8]:
# Trying to silence warnings from polyglot to no avail. They clutter the output, unfortunately.
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [9]:
# This is the core cell for producing the dictionaries and jsons. Edit YEAR and then run this cell.
YEAR = "2005"
d = produce_year_dict(YEAR)
output_dict_to_json(YEAR, d)

Opening corpus for year 2005...
Dataset already exists at /home/lucas/.convokit/downloads/wikiconv-2005


MemoryError: 

2003:
- total utterances: 140265
- total count of code switch: 493
- total count of English switched to other languages: 413
- code switch dictionary: {'Greek': 88, 'Chinese': 82, 'Japanese': 56, 'Polish': 19, 'Dutch': 7, 'Uzbek': 1, 'Finnish': 4, 'xx': 2, 'Swedish': 6, 'Malay': 17, 'French': 28, 'Thai': 4, 'Volapük': 2, 'Basque': 1, 'Hebrew': 5, 'German': 29, 'Norwegian': 2, 'Latin': 6, 'Cebuano': 1, 'Hausa': 1, 'Korean': 7, 'Italian': 2, 'Yiddish': 1, 'Spanish': 5, 'Danish': 13, 'Romanian': 5, 'Lao': 1, 'Mongolian': 1, 'Indonesian': 2, 'Galician': 2, 'Manx': 1, 'Romansh': 1, 'Arabic': 7, 'Sanskrit': 1, 'Esperanto': 1, 'Hindi': 1, 'Croatian': 3, 'Portuguese': 4, 'Russian': 2, 'Maltese': 1, 'Luxembourgish': 1, 'Latvian': 1, 'Norwegian Nynorsk': 3, 'Scots': 2, 'Waray': 2, 'Icelandic': 1, 'Bosnian': 3, 'Bangla': 1, 'Serbian': 2, 'un': 1, 'Tatar': 1, 'zzp': 1, 'Guarani': 2, 'Kinyarwanda': 1, 'Hungarian': 1, 'Slovenian': 1, 'Welsh': 1}
- topics dictionary: {"Liouville's theorem (Hamiltonian)": {'Greek': 1}, 'Names of China': {'Chinese': 1}, 'Michael Hardy/Archive2': {'Greek': 1}, 'Shenzhou 5': {'Japanese': 1}, 'List of Greek phrases': {'Greek': 1}, 'Traditional Chinese characters': {'Chinese': 1}, 'Koszalin': {'Polish': 2}, 'Old Italic script': {'Greek': 1}, 'Kanji': {'Chinese': 1}, 'Bohr magneton': {'Greek': 1}, 'Chinese era name': {'Japanese': 1}, 'Perejil Island': {'Dutch': 1}, 'Duration (music)': {'Greek': 1}, 'Gpoduval': {'Uzbek': 1, 'Finnish': 1}, 'Bopomofo': {'xx': 1}, 'Tao Te Ching': {'xx': 1, 'Chinese': 1}, 'Hwarang': {'Chinese': 1}, 'Cimon Avaro/archive 4': {'Swedish': 1}, 'MartinHarper': {'Malay': 1}, 'Transliteration': {'Japanese': 1, 'Greek': 1}, 'Naming conventions (Chinese)': {'Chinese': 1}, 'Week': {'Chinese': 1}, 'Cultural Revolution/Archive 1': {'Chinese': 1}, 'Korean language/Archive 1': {'Chinese': 2}, 'Dan Koehl': {'Swedish': 1}, 'Pi': {'French': 1}, 'Shawnyhc01': {'Chinese': 1}, 'Main Page/Archive 6': {'Polish': 1}, 'Whampoa Military Academy': {'Chinese': 1}, 'Amnat Charoen Province': {'Thai': 1}, 'KTDykes': {'Volapük': 1}, 'Egon': {'Basque': 1}, 'Baphomet': {'Hebrew': 1}, 'Wernher Von Braun': {'German': 1}, 'Greek alphabet': {'Greek': 1, 'Volapük': 1}, 'Jan Simonsen': {'Norwegian': 1}, 'Shellsort': {'German': 1}, 'Operating system': {'Latin': 1}, 'Meizhou Island': {'Chinese': 1}, 'Jurriaan Schulman': {'Dutch': 4}, 'Court of Cassation (France)': {'French': 1}, 'Canon Inc.': {'Japanese': 1}, 'Shizhao': {'Chinese': 1}, 'Padri War': {'French': 1}, 'Ying Nianzi': {'Chinese': 1}, 'Ericd/Archives 2': {'French': 1}, 'Ed Poor': {'Chinese': 1}, 'Brion VIBBER': {'Chinese': 1}, 'Federation of Expellees': {'German': 1}, 'Palindrome': {'Chinese': 1, 'Japanese': 1}, 'Comparison of American and British English': {'Greek': 1}, 'Japanese proverbs': {'Japanese': 1}, 'Yanyang1985': {'Hausa': 1}, 'Yalu River': {'Korean': 1}, 'Hong Kong Economic Journal': {'Japanese': 1}, 'Manual of Style/Layout': {'Italian': 1}, 'C (programming language)': {'Japanese': 1}, 'Haredi Judaism': {'Hebrew': 1, 'Yiddish': 1}, 'Martingale (probability theory)': {'Greek': 1}, 'Eclipse cycle': {'Greek': 1}, 'Danny/Archive 2': {'Hebrew': 1}, 'Youssefsan': {'French': 1}, 'Estimator': {'Greek': 1}, 'Rank-into-rank': {'Greek': 1}, 'Tongyong Pinyin': {'Chinese': 1}, 'Pablo.cl': {'Spanish': 1}, 'Mong Kok': {'Japanese': 1}, 'Constellation': {'Greek': 1}, 'Japanese era name': {'Korean': 1, 'Danish': 1}, 'Dimitrie Cantemir': {'Romanian': 1}, 'Callisthenes': {'Greek': 1}, 'Samuel~enwiki': {'Japanese': 1}, 'WikiProject Constellations': {'Danish': 1}, 'Demchugdongrub': {'Chinese': 1}, 'A very elementary proof that 22/7 exceeds π': {'Greek': 1}, 'Taoster': {'Chinese': 2}, 'Lao language': {'Lao': 1}, 'Cup of Solid Gold': {'Chinese': 1}, 'Grammatical particle': {'German': 1}, 'Mainland China/Archive 1': {'Chinese': 1}, 'AntonioMartin': {'Spanish': 1}, 'Regions of Finland': {'Finnish': 1}, 'King Wu of Zhou': {'Chinese': 1}, '2channel': {'Japanese': 1}, 'Ktsquare/Archive I': {'Chinese': 1}, 'Ericd/Archives - Discussions that were on my user page': {'French': 1}, 'Menchi/Stardate 0.0004': {'Indonesian': 1, 'Chinese': 1}, 'Sagittarius (constellation)': {'Greek': 1}, 'TakuyaMurata/Takubot4': {'Danish': 1, 'Japanese': 1}, 'Tokerboy~enwiki/archive': {'Malay': 1}, 'Golden ratio base': {'Greek': 1}, 'Chiba Prefecture': {'Japanese': 1}, 'National Congress of the Communist Party of China': {'Chinese': 1}, 'Passerine': {'Latin': 1, 'Galician': 1}, 'Chen Shui-bian': {'Chinese': 1, 'Danish': 1}, 'Kingturtle': {'Greek': 1}, 'Kosebamse/Archive1': {'German': 1}, 'Portuguese language': {'Galician': 1}, 'Jimbo Wales': {'German': 1}, 'List of French monarchs/Archive 4': {'French': 1}, 'Combinatory logic': {'Greek': 1}, 'Xaos~enwiki': {'Korean': 1}, 'Kangxi Emperor': {'Chinese': 1}, 'Bytów': {'Polish': 1}, 'Tokyo': {'Japanese': 1}, 'Vergina~enwiki': {'Manx': 1, 'Romansh': 1}, 'American Chinese cuisine': {'Chinese': 1}, 'Djmutex/old002': {'German': 1}, 'Wards of Japan': {'Danish': 1}, 'Romanian language': {'Romanian': 1}, 'ß': {'German': 1}, 'Main Page': {'Polish': 1}, 'Rogper~enwiki/2003-2007': {'Swedish': 1}, 'Diaspora': {'Dutch': 1}, 'Emperor Jimmu': {'Japanese': 1}, 'Provinces of Finland': {'Swedish': 1}, 'Historical Chinese anthems': {'Chinese': 1}, 'Charles Matthews': {'Greek': 1}, 'Orestes': {'German': 1}, 'Amish': {'Japanese': 1}, 'Carbuncle': {'Japanese': 1}, 'Two16': {'Malay': 1}, 'Adrienne Clarkson': {'Chinese': 1}, 'Sopot': {'Polish': 1}, 'Supernatural/Archive 3': {'Greek': 1}, 'Formulax~enwiki': {'Japanese': 1, 'Chinese': 2}, 'Mathematics': {'Greek': 1}, 'Logical fallacy': {'Greek': 1}, 'Group sex': {'Japanese': 1}, 'Severe acute respiratory syndrome': {'Chinese': 1}, 'Urho Kekkonen': {'Finnish': 1}, 'Esperanto': {'Esperanto': 1}, 'Kaspar Hauser': {'German': 1}, 'Chinese characters': {'Chinese': 1}, 'Djmutex/old001': {'German': 1}, 'Araisyohei': {'Japanese': 1}, 'Ilonggo': {'Malay': 1}, 'Anti-French sentiment in the United States/archive 6': {'French': 1}, 'LittleDan': {'Greek': 1}, 'Menchi/Stardate 0.0001': {'Chinese': 1}, "Lambert's W function": {'Greek': 1}, 'JeLuF': {'German': 1}, 'West Bank': {'Hebrew': 1}, 'Ley line/Ley lines and probability': {'Greek': 1}, 'Rlvaughn': {'Greek': 1}, 'Cimon Avaro/archive3': {'Finnish': 1}, "Goldbach's conjecture": {'Greek': 1}, 'Stevertigo/Archive 2': {'Malay': 1}, 'Mosstoh': {'Chinese': 1}, 'Kościerzyna': {'Polish': 1}, 'Open set': {'Greek': 1}, 'Cao Rui': {'Chinese': 1}, 'Ecoregion conservation status': {'French': 1}, 'Image use policy/Archive 4': {'French': 1}, 'Hymn to Liberty': {'Greek': 1}, 'Gyan': {'Hindi': 1}, 'Goguryeo': {'Japanese': 1}, 'List of political parties in Croatia': {'Croatian': 1}, 'WikiMoney': {'Greek': 2}, 'Pauli matrices': {'Greek': 1}, 'AxelBoldt/Archive December 2004 - December 2006': {'Japanese': 1}, 'Utility': {'Greek': 1}, 'Katakana': {'Japanese': 1}, 'PedroPVZ': {'Portuguese': 1}, 'Stevertigo/Archive 4': {'Russian': 1, 'Chinese': 1}, 'Wanli': {'Chinese': 1}, "Hilbert's sixteenth problem": {'German': 1}, 'Ø': {'Greek': 1}, 'Mecca': {'Arabic': 1}, 'Theseus': {'Greek': 1}, 'Nazism': {'German': 1}, 'Hanns Eisler': {'Luxembourgish': 1}, 'Michaud Affair': {'French': 1}, 'Ericd/Archives 1': {'French': 1}, 'Franjo Tuđman': {'Croatian': 1}, 'Transcendental number': {'Greek': 1}, 'Situationist International': {'Italian': 1}, 'Akhenaten': {'Spanish': 1}, 'Białogard': {'Polish': 1}, 'Fwappler': {'German': 1}, 'Pierre Teilhard de Chardin': {'French': 1}, 'WikiProject Countries/Proposal 1': {'Japanese': 1, 'Chinese': 1}, 'List of Naval Vessels of World War II': {'German': 1}, 'Yi Sun-sin': {'Japanese': 1, 'Korean': 1}, 'Clitic': {'Greek': 1}, 'Multivariate normal distribution': {'Greek': 1}, 'Toby Bartels': {'Greek': 1}, 'Mav/archive 15': {'German': 1}, 'Chinese white dolphin': {'Chinese': 1}, "Chaitin's constant": {'Greek': 1}, 'Likelihood-ratio test': {'Greek': 1}, 'Chien-Shiung Wu': {'Chinese': 1}, 'Penang': {'Chinese': 1}, 'Daifang Commandery': {'Chinese': 1}, 'Cluj-Napoca Botanical Garden': {'Romanian': 1}, 'Chaos theory/Archive 5': {'Portuguese': 1}, 'Stevertigo': {'Malay': 1}, 'Yasothon Province': {'Thai': 1}, 'Nanshu/Archive001': {'Chinese': 1}, 'TakuyaMurata/Takubot3': {'Danish': 1}, 'Okinawa Prefecture': {'Japanese': 1}, 'Silesia/Archive 1': {'Polish': 1, 'German': 1}, 'Proof that 22/7 exceeds π': {'Greek': 1}, 'Jus sanguinis': {'Portuguese': 1}, 'Camembert': {'Malay': 1}, 'Scientific method': {'French': 1}, 'Anting': {'Chinese': 1}, 'Tarquin/Archive 3': {'French': 1}, 'Silesia/Archive 5': {'Polish': 1}, 'Mount Athos': {'Greek': 1}, 'Mkweise': {'Japanese': 1}, 'Talsi': {'Latvian': 1}, 'Ortolan88/Archive1': {'Malay': 1}, 'TakuyaMurata': {'Danish': 1}, 'Regional accents of English': {'Chinese': 1}, 'List of Polish proverbs': {'Polish': 1}, 'Axel Munthe': {'German': 1}, '24.127.3.230': {'Norwegian Nynorsk': 1}, 'Marsaxlokk': {'French': 1}, 'Media of Hong Kong': {'Japanese': 1}, 'Zhongshan': {'Chinese': 2}, 'Ley line': {'Greek': 1}, 'Loess': {'German': 1}, 'Djmutex': {'German': 1}, 'Chanthaburi Province': {'Thai': 1}, 'Menchi/Stardate 0.0003': {'Chinese': 3}, 'Subobject classifier': {'Greek': 1}, 'Housewife': {'Japanese': 1}, 'Generation name': {'Danish': 1, 'Japanese': 1}, 'Empress Gi': {'Chinese': 1}, 'Tokerboy~enwiki': {'Malay': 1}, 'Mintguy/archive 1': {'Malay': 1}, 'Groundhog Day': {'Scots': 1}, 'Jdstroy': {'Portuguese': 1, 'Waray': 1}, 'Poor metal/Old article copy': {'Latin': 1}, 'TomeRaider database': {'Greek': 1}, 'EPR paradox': {'Greek': 1}, 'Vera Cruz': {'Malay': 1}, 'Rumia': {'Polish': 1}, 'List of Swedish monarchs': {'Icelandic': 1}, 'TUF-KAT': {'Japanese': 1}, 'Nara period': {'Japanese': 1}, 'History of Bosnia and Herzegovina': {'Bosnian': 1}, 'Minimal pair': {'Greek': 1}, 'Mpt/Weblog': {'Danish': 1, 'Korean': 1}, 'Menchi/Stardate 0.0002': {'Chinese': 1}, 'Yōjirō Ishizaka': {'Japanese': 1}, 'Balhae': {'Chinese': 1}, 'Casimir effect': {'Greek': 1}, 'Hippietrail': {'Bangla': 1}, 'Roman emperor': {'Latin': 1}, 'Archimedes': {'Greek': 1}, 'Lorenzarius': {'Japanese': 1}, 'Fubar Obfusco': {'Japanese': 1}, 'Royal house': {'German': 1, 'Waray': 1}, 'Heresy': {'Greek': 1}, 'Xiongnu': {'Japanese': 1}, 'Viajero/Archive 2': {'Spanish': 1}, 'Durrës': {'Romanian': 1}, 'Szczecinek': {'Polish': 1}, 'Omega': {'Greek': 1}, 'Zoe': {'Japanese': 1}, 'Hangul/Archive 1': {'Korean': 1}, 'Ktsquare': {'Chinese': 1}, 'Poznań': {'Polish': 1, 'German': 1}, 'Gdańsk/Archive 1': {'Polish': 1}, 'Chinese literature': {'Chinese': 1}, 'Dinopup': {'Greek': 1}, 'Angela/Archive1': {'Arabic': 1}, 'Erichsu': {'Chinese': 1}, "Modelling Wikipedia's growth": {'Greek': 1}, 'Virgin birth of Jesus/Archive 1': {'Greek': 1}, 'Bukkake': {'Japanese': 1}, 'Eloquence': {'Japanese': 1, 'Scots': 1}, 'Kosovo (region)/Archive 1': {'Serbian': 1}, 'Angela/Archive5': {'Arabic': 1}, 'Irish language': {'Greek': 1}, 'Momentum': {'Greek': 1}, 'Glossary of French expressions in English': {'French': 1}, 'Serbo-Croatian/Archive 2': {'Croatian': 1}, 'Rob Hooft': {'Dutch': 1}, 'History of Poland (1939–1945)': {'Polish': 1}, 'Calendar date': {'Chinese': 1, 'Japanese': 1}, 'Gauss–Markov theorem': {'Greek': 1}, 'Manual of Style': {'Japanese': 1}, 'Kartuzy': {'Polish': 1}, 'Japanese cuisine': {'Japanese': 1}, 'Orkhon inscriptions': {'un': 1, 'Japanese': 1}, 'Mathieugp': {'French': 1}, 'Tarquin/Archive 5': {'French': 1}, 'Slrubenstein/archive 1': {'Malay': 1}, 'Anthere/Archives': {'French': 1}, 'Olga Bityerkokoff': {'Russian': 1}, 'Geats': {'Swedish': 1}, 'Ridetheory': {'Japanese': 1}, 'Greeklish': {'Greek': 1}, 'Kobe': {'Japanese': 1}, "San'yō Shinkansen": {'Japanese': 1}, 'Croat and Bosnian neologisms': {'Bosnian': 1}, 'Maria Theresa': {'German': 1}, 'Kosovo/Archive 5': {'Serbian': 1}, 'Taxicab': {'Greek': 1}, 'Silesia/Archive 7': {'Polish': 1}, 'Koyaanis Qatsi': {'French': 1}, 'Stevertigo/Archive 6': {'Japanese': 1}, 'Yukos': {'Tatar': 1}, 'Naming conventions (Chinese)/Archive 2': {'Chinese': 1}, 'Dangun': {'Korean': 1}, 'Confucianism': {'Chinese': 1}, 'Leslie Cheung': {'Chinese': 1}, 'Bcorr/Archive 200310': {'French': 1}, 'Swedish alphabet': {'Danish': 1, 'zzp': 1}, 'º¡º': {'Japanese': 1, 'Chinese': 1}, 'Only make links that are relevant to the context/Archive 1': {'Latin': 1}, 'Folk music': {'Danish': 1, 'Indonesian': 1}, 'Chiang Ching-kuo': {'Japanese': 1}, 'Lorem ipsum': {'Latin': 1}, 'Sam Francis': {'Malay': 1}, 'Manual of Style/Biography': {'Japanese': 1}, 'Tczew': {'Polish': 1}, 'Kara-Khanid Khanate': {'Chinese': 1}, 'Sheaf (mathematics)': {'Greek': 1}, 'Big O notation': {'Greek': 1}, 'Dictator': {'Kinyarwanda': 1, 'Spanish': 1}, 'Kyushu': {'Japanese': 1}, 'Norway': {'Norwegian': 1}, 'Comparison of American and British English/Archive 1': {'Danish': 1}, 'Zoe/archive 7': {'Malay': 1}, 'Viscosity': {'Greek': 1}, 'Electromagnetism': {'Greek': 1}, "Spearman's rank correlation coefficient": {'Greek': 1}, 'Hungary': {'Hungarian': 1}, 'Slovene language': {'Slovenian': 1}, 'Heligoland': {'Welsh': 1}, 'Breisgau-Hochschwarzwald': {'German': 1}, 'Covariance matrix': {'Greek': 1}, 'Flauto Dolce': {'Greek': 1}, 'Mainland Japan': {'Japanese': 1}, 'Emperor of Japan': {'Chinese': 2}, 'Flight and expulsion of Germans (1944–50)': {'German': 1}, 'Axiom': {'Greek': 1}, 'List of French monarchs/Archive 2': {'French': 1}, 'Bcorr/Archive 200311': {'French': 1}, 'Ranks of nobility and peerage': {'German': 1}, 'Contradiction': {'German': 1}, 'Slobodan Milošević': {'Bosnian': 1}, 'Sieve of Eratosthenes': {'Greek': 1}, 'Akihito': {'Chinese': 1}, 'Satrap': {'Greek': 1}, 'David.Monniaux/archive1': {'French': 1}, 'Function specification (mathematics)': {'Greek': 1}, 'Historical archive/Sandbox': {'German': 1}}